In [1]:
!pip install paho-mqtt

     |████████████████████████████████| 99 kB 2.7 MB/s 
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62133 sha256=64c50cf28a5f3c89bfd259eb95ceefc830f5401c1bc224e09be5a5698e831f72
  Stored in directory: /root/.cache/pip/wheels/d0/bf/ac/2b3f43f8c6fcd0f4ba5395397458c521eb0b52d33b574a5a40
Successfully built paho-mqtt


In [2]:
!pip install pandas-profiling -U

     |████████████████████████████████| 261 kB 7.5 MB/s 
     |████████████████████████████████| 675 kB 42.6 MB/s 
     |████████████████████████████████| 596 kB 46.3 MB/s 
     |████████████████████████████████| 303 kB 55.8 MB/s 
     |████████████████████████████████| 102 kB 12.3 MB/s 
     |████████████████████████████████| 62 kB 873 kB/s 
     |████████████████████████████████| 3.1 MB 32.1 MB/s 
     |████████████████████████████████| 10.1 MB 30.3 MB/s 
     |████████████████████████████████| 812 kB 34.7 MB/s 
     |████████████████████████████████| 38.1 MB 2.1 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=5717d2da5187aaaa333d471c048a4d2bd4fbba668f83f81cb9368223aa19de39
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295207 sha256=0fe9b0c40b547133897752e93d826ae5c37aba402e295d632f6d2756ea41ef79
 

In [3]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 8.1 MB/s 
     |████████████████████████████████| 895 kB 22.8 MB/s 
     |████████████████████████████████| 3.3 MB 30.8 MB/s 
     |████████████████████████████████| 61 kB 495 kB/s 


In [4]:
!pip install feature_engine -U

     |████████████████████████████████| 180 kB 7.8 MB/s 
     |████████████████████████████████| 9.8 MB 49.5 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
import os
import re
import csv

import torch
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

from tqdm import tqdm

import plotly.express as px
import paho.mqtt.client as mqtt
import matplotlib.pyplot as plt
from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D
from pandas_profiling import ProfileReport

from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

import xgboost as xgb
import lightgbm as lgb

from yellowbrick.regressor import ResidualsPlot

from mlxtend.regressor import StackingCVRegressor

from transformers import get_linear_schedule_with_warmup

sns.set_theme(style="darkgrid")

%matplotlib inline
random_state = 42

In [ ]:
host = 'iot.maratona.dev'
port = 31666
username = 'maratoners'
password = 'btc-2021'
topic = 'quanam'

In [ ]:
def on_message(client, userdata, message):
    global recieved_data
    recieved_data = str(message.payload.decode("utf-8"))
    print("%s %s" % (message.topic, message.payload))
    # Write the returned messages into a csv file
    csv_writer.writerow([recieved_data])
    my_data_file.flush()   

client = mqtt.Client(client_id='', 
                        clean_session=True, 
                        userdata=dict, 
                        protocol=mqtt.MQTTv311, 
                        transport='tcp')

client.username_pw_set(username=username, password=password)

client.connect(host=host, port=port)

client.loop_start()
client.subscribe(topic)

my_data_file = open('quanam.csv', 'a+')
csv_writer = csv.writer(my_data_file, delimiter=',')

# Iterate over all elements until reach the first element one more time 
# and then stop the run of the cell manually
while True:
    client.on_message = on_message

client.disconnect()
client.loop_stop()

In [ ]:
with open('quanam.csv', 'r') as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]

In [ ]:
splited_lines = []

for line in tqdm(lines):
    # Get values of each line
    regex_found = re.findall('""[A-Z]+[0-9]*"":\s"*"*([0-9]+\.*[0-9]*)"*"*', line)
    # Assert that the right number of values is returned
    assert len(regex_found) == 7 or line!='', f"{len(regex_found)}"
    
    splited_lines.append(np.array(regex_found).astype(np.float))

In [ ]:
df = pd.DataFrame(splited_lines, columns=['ILLUM', 'HUMID', 'CO2', 'SOUND','TEMP','RYTHM', 'ID'])
df

In [ ]:
df.info()

In [ ]:
df = df.drop_duplicates()
df = df.dropna()
df = df.sort_values(by = 'ID').reset_index()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
profile = ProfileReport(df)

In [ ]:
profile

In [ ]:
rcParams['figure.figsize'] = 14,8

In [ ]:
vars = df.drop(['df_index', 'ID'], axis=1).columns

In [ ]:
for var in vars:
    sns.regplot(data=df, x="RYTHM", y=var)
    plt.show()

In [ ]:
correlation_matrix = df[vars].corr().round(2)
figure = plt.figure(figsize=(12, 12))
sns.heatmap(data=correlation_matrix, annot=True)

In [ ]:
sns.regplot(data=df, x="SOUND", y='HUMID')

In [ ]:
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111, projection = '3d')

x = df['CO2']
y = df['TEMP']
z = df['RYTHM']

ax.set_xlabel("CO2")
ax.set_ylabel("TEMP")
ax.set_zlabel("RYTHM")

ax.scatter(x, y, z)
plt.show()

In [ ]:
fig = px.scatter_3d(df, x='CO2', y='TEMP', z='RYTHM')
fig.show()

In [ ]:
def check_distribution(df, variable):
    plt.figure(figsize=(16, 4))

    # histogram
    plt.subplot(1, 3, 1)
    sns.histplot(df[variable], bins=30)
    plt.title('Histogram')

    # Q-Q plot
    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.ylabel('Variable quantiles')

    plt.show()

In [ ]:
for var in vars:
    check_distribution(df, var)
    plt.show()

In [ ]:
# Check for outliers 
for var in vars:
    sns.boxplot(y=df[var])
    plt.show()

In [ ]:
y_train = df['RYTHM']
X_train = df.drop(['RYTHM', 'df_index', 'ID', 'HUMID'], axis=1)

In [ ]:
X_train.info()

In [ ]:
y_train

In [ ]:
LR_pipe_scaler = Pipeline([
    # feature Scaling - section 10
    ('scaler', RobustScaler()),
    
    # regression
    ('LR', LinearRegression())
])

lasso_pipe_scaler = Pipeline([
    # feature Scaling - section 10
    ('scaler', RobustScaler()),
    
    # regression
    ('lasso', Lasso(alpha=0.0165, random_state=random_state))
])

ENet_pipe_scaler = Pipeline([
    # feature Scaling - section 10
    ('scaler', RobustScaler()),
    
    # regression
    ('ENet', ElasticNet(alpha=0.0085, l1_ratio=0.1, random_state=random_state))
])

KRR_pipe_scaler = Pipeline([
    # feature Scaling - section 10
    ('scaler', RobustScaler()),
    
    # regression
    ('krr', KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5))
])

model_lgb = Pipeline([
    # feature Scaling - section 10
    ('scaler', RobustScaler()),
    
    # regression
    ('lgb', lgb.LGBMRegressor(objective='regression',
                              learning_rate=0.1,
                              max_depth=-1,
                              n_estimators=200,
                              num_leaves=2,
                              random_state=random_state))
])


gbr = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state=random_state)

xgboost = xgb.XGBRegressor(learning_rate=0.01, n_estimators=3000,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                    objective='reg:squarederror',
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006, random_state=random_state)

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    # Source: https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (LR_pipe_scaler, lasso_pipe_scaler, ENet_pipe_scaler, KRR_pipe_scaler, model_lgb),
                                                    meta_model = lasso_pipe_scaler)

In [ ]:
for clf, label in zip([LR_pipe_scaler, lasso_pipe_scaler, ENet_pipe_scaler, KRR_pipe_scaler, model_lgb, xgboost, gbr, stacked_averaged_models], ['LR', 'Lasso', 
                                                'ENet', 
                                                'KRR', 'LGB', 'xgboost', 'gbr','Stack']):
    scores = cross_val_score(clf, X_train.values, y_train.values, cv=5, scoring='r2')
    print("R^2 Score: %0.6f (+/- %0.6f) [%s]" % (
        scores.mean(), scores.std(), label))

In [ ]:
stacked_averaged_models.fit(X_train.values, y_train.values)
stacked_train_pred = stacked_averaged_models.predict(X_train.values)
print(r2_score(y_train, stacked_train_pred))

In [ ]:
answers_df = pd.read_csv('answers.csv')

In [ ]:
answers_df.head()

In [ ]:
profile = ProfileReport(answers_df)

In [ ]:
profile

In [ ]:
ans_df_alt = answers_df.drop(['RYTHM', 'ID', 'HUMID'], axis=1)

In [ ]:
ans_df_alt.head()

In [ ]:
ans_pred = stacked_averaged_models.predict(ans_df_alt.values)

In [ ]:
len(ans_pred)

In [ ]:
my_answers_df = pd.read_csv('answers.csv')

In [ ]:
my_answers_df.info()

In [ ]:
my_answers_df = my_answers_df.drop(['RYTHM'], axis=1)

In [ ]:
my_answers_df.head()

In [ ]:
my_answers_df['RYTHM'] = ans_pred

In [ ]:
my_answers_df

In [ ]:
my_answers_df.to_csv('MY_ANSWERS_DEF_2.csv',index=False)